In [1]:
import re
import numpy as np
import pandas as pd
from collections import Counter
from scrabble_assistant import read_json
from dictionary_preprocessing import calculate_word_value
from dictionary_preprocessing import is_word_correct
from dictionary_preprocessing import is_word_available
from dictionary_preprocessing import get_regex_patterns
from scrabble_assistant import get_marked_rows

# Считываем словарь

In [2]:
rus_words_data = pd.read_csv('dictionary.txt', 
                             header=None,
                             names=['word'])
rus_words_data

,word
0,абажур
1,абаз
2,абазин
3,абазинка
4,абака
...,...
34005,ящурка
34006,ёкание
34007,ёлка
34008,ёмкость


# Добавляем колонку с длиной слова

In [3]:
rus_words_data['length'] = rus_words_data.word.apply(lambda x: len(x))
rus_words_data

,word,length
0,абажур,6
1,абаз,4
2,абазин,6
3,абазинка,8
4,абака,5
...,...,...
34005,ящурка,6
34006,ёкание,6
34007,ёлка,4
34008,ёмкость,7


# Убираем слова длинее 15 букв

In [4]:
rus_words_data = rus_words_data.query('length <= 15')
rus_words_data

,word,length
0,абажур,6
1,абаз,4
2,абазин,6
3,абазинка,8
4,абака,5
...,...,...
34005,ящурка,6
34006,ёкание,6
34007,ёлка,4
34008,ёмкость,7


# Убираем слова с неожиданными символами и содержащие больше букв, чем есть в игре

In [5]:
rus_words_data['is_correct'] = rus_words_data.word.apply(lambda x: is_word_correct(x))
rus_words_data = rus_words_data.query('is_correct == True')
rus_words_data = rus_words_data.drop('is_correct', axis=1) # больше не нужна
rus_words_data

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,word,length
0,абажур,6
1,абаз,4
2,абазин,6
3,абазинка,8
4,абака,5
...,...,...
34001,ящер,4
34002,ящерица,7
34003,ящик,4
34004,ящур,4


# Вы не видели никаких предупреждений. Блокнот идеален.
<img alt="Медики научились стирать память | Новые Известия" class="n3VNCb" src="https://newizv.ru/attachments/317af9c89fc354109c47c029ed16c501e5db3bb5/store/fill/1200/675/b8d575d01c39e48984e1c38e7749fb2cb856c4c95c1aec1038be76212e09/b8d575d01c39e48984e1c38e7749fb2cb856c4c95c1aec1038be76212e09.jpg" data-noaft="1" jsname="HiaYvf" jsaction="load:XAeZkd;" style="width: 434px; height: 244.125px; margin: 0px;">

In [6]:
rus_words_data['value'] = rus_words_data.word.apply(lambda x: calculate_word_value(x))
rus_words_data

,word,length,value
0,абажур,6,15
1,абаз,4,10
2,абазин,6,12
3,абазинка,8,15
4,абака,5,8
...,...,...,...
34001,ящер,4,16
34002,ящерица,7,28
34003,ящик,4,16
34004,ящур,4,18


# Отранжируем слова по цене (без учета бонусных полей на доске).

In [7]:
rus_words_data.sort_values(by='value', ascending=False)

,word,length,value
32910,шлифовальщица,13,56
32862,шифровальщица,13,56
33274,экзаменующаяся,14,54
32918,шлихтовальщица,14,53
2370,брошюровщица,12,53
...,...,...,...
6926,ер,2,3
17137,ом,2,3
885,ар,2,3
299,ад,2,3


# Найдем все слова, которые можем составить с заданным набором букв.

In [8]:
rus_words_data['available'] = rus_words_data.word.apply(lambda x: is_word_available(Counter('абвгдеё'), x))
rus_words_data.query('available == True')

,word,length,value,available
299,ад,2,3,True
1471,бег,3,7,True
2786,веда,4,6,True
6868,еда,3,4,True


# Из букв 'абвгдеё' можно составить вышеперчисленные слова.

# Тест функций get_marked_rows и get_regex_patterns:

In [9]:
test_board = [
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', 'т', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', 'о', '', '', '', '', ''],
        ['', '', '', 'п', 'о', 'c', 'е', 'л', 'о', 'к', '', '', '', '', ''],
        ['', '', '', 'а', '', 'а', '', '', '', '', '', 'р', '', '', ''],
        ['', '', '', 'п', '', 'д', 'о', 'м', '', 'я', '', 'е', '', '', ''],
        ['', '', '', 'а', '', '', '', 'а', 'з', 'б', 'у', 'к', 'а', '', ''],
        ['', '', '', '', '', 'с', 'о', 'м', '', 'л', '', 'а', '', '', ''],
        ['', '', '', 'я', 'м', 'а', '', 'а', '', 'о', '', '', '', '', ''],
        ['', '', '', '', '', 'л', '', '', '', 'к', 'и', 'т', '', '', ''],
        ['', '', '', '', 'с', 'о', 'л', 'ь', '', 'о', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
    ]

In [10]:
test_row_after_sharping = get_marked_rows(test_board)[5]
test_row_after_sharping

['', '', '', 'а', '#', 'а', '#', '#', '#', '#', '', 'р', '', '', '']

In [11]:
patterns = get_regex_patterns(test_row_after_sharping)
patterns

['^([а-я]?[а-я]?[а-я]?а)$', '^([а-я]?р[а-я]?[а-я]?[а-я]?)$']

In [12]:
rus_words_data.word.str.extract('^([а-я]{,3}а)$', flags=re.UNICODE).dropna()

,0
379,айва
903,арба
946,арка
985,арфа
1022,арча
...,...
33919,яйла
33938,яма
33957,ярка
33991,яхта


In [14]:
for pattern in patterns:
    print(rus_words_data.word.str.extract(pattern, flags=re.UNICODE).dropna())

          0
379    айва
903    арба
946    арка
985    арфа
1022   арча
...     ...
33919  яйла
33938   яма
33957  ярка
33991  яхта
34000  яшма

[192 rows x 1 columns]
           0
885       ар
886     араб
892     арак
899     арат
903     арба
...      ...
33874  юрист
33880   юрок
33881   юрта
33956    ярд
33957   ярка

[290 rows x 1 columns]
